In [1]:
# Math manipulation
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
import numpy as np
import math

# Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import pyplot
from matplotlib.patches import Rectangle
%matplotlib inline

# Image processing
import cv2
from io import BytesIO

# Deep Learning
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from keras.preprocessing.image import img_to_array
from mrcnn.visualize import random_colors, apply_mask, find_contours, patches, Polygon  # Export workarround

# Utils
import os
import math

Using TensorFlow backend.
/home/benitez/anaconda3/envs/devCPU/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/benitez/anaconda3/envs/devCPU/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/benitez/anaconda3/envs/devCPU/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/benitez/ana

In [2]:
class Flags:
    deleteWeights = False
    downloadWeights = False
flags = Flags()

# Instance segmentation definition

I'm usign RCNN arquitecture pre trained on MSCOCO dataset


results = dictionary for each image that we passed do the detect()

results[0] = prediction of the first image (in our case, th only one)

The keys of the dictionary of note are as follows:

* ‘rois‘: The bound boxes or regions-of-interest (ROI) for detected objects, bottom left and top right (y1, x1, y2, x2), in pixels
* ‘masks‘: The masks for the detected objects.
* ‘class_ids‘: The class integers for the detected objects.
* ‘scores‘: The probability or confidence for each predicted class

In [3]:
weightPath = 'models/mask_rcnn_coco.h5'
if flags.downloadWeights==True: 
    !wget -O $weightPath https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [10]:
def _display_instances(image, boxes, masks, class_ids, class_names,
                    scores=None, title="",
                    skip=False,
                    figsize=(16, 16), ax=None,
                    show_mask=True, show_bbox=True,
                    colors=None, captions=None, verbose=True, save_img=False, image_path=""):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    _, ax = plt.subplots(1, figsize=figsize)

    # Generate random colors
    colors = colors or random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        if class_ids[i]==1: color=(1.0, 0.0, 0.0)
        elif skip: continue#color = (0.0,0.0,0.0)
        else: color = (0.1,0.1,0.6)
        #color = colors[i]
        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                            alpha=0.7, linestyle="dashed",
                            edgecolor=color, facecolor='none')
        ax.add_patch(p)
        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        ax.text(x1, y1 + 8, caption, color='w', size=11, backgroundcolor="none")
        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = apply_mask(masked_image, mask, color)
        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
    ax.imshow(masked_image.astype(np.uint8))
    if save_img:
        plt.savefig(image_path,bbox_inches='tight', pad_inches=-0.5,orientation= 'landscape')
    if verbose:
        plt.show()
    buff = BytesIO()
    plt.savefig(buff, format="png", bbox_inches='tight', pad_inches=-0.1, orientation= 'landscape')
    buff.seek(0)
    
    plt.close('all')
    return buff

In [5]:
# define the custom configuration (the other variables have good default values)
class config(Config):
    NAME = "barnInstanceSegmentation"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 80
    DETECTION_MIN_CONFIDENCE = 0.55
    DETECTION_NMS_THRESHOLD = 0.1
    DETECTION_MAX_INSTANCES = 150
    #I think that the input size can be modified
    
# define 81 classes that the coco model knowns about
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'cow', 'cow', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# define the model
rcnn = MaskRCNN(mode='inference', model_dir='./models/', config=config())
# load coco model weights
rcnn.load_weights(weightPath, by_name=True)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [6]:
def instanceSegmentation(frame, save_img=True, image_path='foo.jpg'):
    results = rcnn.detect([frame], verbose=0)
    # show photo with bounding boxes, masks, class labels and scores
    img = _display_instances(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 
                                results[0]['rois'], 
                                results[0]['masks'], 
                                results[0]['class_ids'],
                                class_names, 
                                results[0]['scores'],
                                #title='Frame instance segmentation',
                                verbose=0,
                                save_img=save_img,
                                image_path=image_path)

# Emotion analysis

Face detection:
https://machinelearningmastery.com/how-to-perform-face-detection-with-classical-and-deep-learning-methods-in-python-with-keras/
    

In [7]:
from mtcnn.mtcnn import MTCNN
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import argparse
import imutils

In [8]:
detector = MTCNN()
model = load_model('models/epoch_60.hdf5')
EMOTIONS = ["angry", "scared", "happy", "sad", "surprised","neutral"]

def emotionAnalysis(frame, file_name):
    result_list = detector.detect_faces(frame)
    for result in result_list:
        # Draw Rectangle
        x1, y1, width, height = result['box']
        x2, y2 = x1+width, y1+height
        frame = cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 2)
        
        # Draw landmarks
        for key, value in result['keypoints'].items():
            frame = cv2.circle(frame, value, 2, (0,0,255), 2)
            
        # Detect emotion
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        roi = gray[y1:y2, x1:x2]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        preds = model.predict(roi)[0]
        i = preds.argmax()
        label = "%.2f"%preds[i] + " " + EMOTIONS[i]
        frame = cv2.putText(frame, label, (x1, y1),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

        #save
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imsave(file_name, frame)

# Run to test videos

In [11]:
videoPath = 'data_samples/Running/'
videoName = 'Running_in_Space00000135'
save_img = True # True=the function above will already save the image, otherwise we save ourself
n=5 # number of frames to take from this video

##################

#os.system('mkdir temp_out')
camera = cv2.VideoCapture(videoPath+videoName+'.mp4')
i=0
total = camera.get(cv2.CAP_PROP_FRAME_COUNT)
interval = int(math.floor(camera.get(cv2.CAP_PROP_FRAME_COUNT)/n))
while True:
    (grabbed, frame) = camera.read()
    i+=1
    if not grabbed: break # end of video
    if i%interval: continue #skip frame
        
    instanceSegmentation(frame, save_img=save_img, image_path='temp_out/'+str(i)+'.jpg')
    #emotionAnalysis(frame, 'temp_out/'+str(i)+'.jpg')
    
    print('--->', 100*i/total, '%')
#os.system('zip -r ' + videoName + ' temp_out')
#os.system('rm -r temp_out')

---> 19.91584852734923 %
---> 39.83169705469846 %
---> 59.747545582047685 %
---> 79.66339410939692 %
---> 99.57924263674614 %
